In [1]:
from elasticsearch import Elasticsearch


        

In [2]:
import requests
import json

url = "https://api.orcid.org/v1.2/search/orcid-bio/?defType=edismax&q=digital-object-ids:%2210.1145/2809563.2809571%22"

headers = {"Content-Type":"application/json",
         "Authorization": "Bearer TOKEN_HERE"}
r=requests.get(url, headers=headers);
data = r.json()
data

{'error-desc': None,
 'message-version': '1.2',
 'orcid-profile': None,
 'orcid-search-results': {'num-found': 1,
  'orcid-search-result': [{'orcid-profile': {'client-type': None,
     'group-type': None,
     'orcid': None,
     'orcid-activities': None,
     'orcid-bio': {'biography': None,
      'contact-details': {'address': {'country': {'value': 'GB',
         'visibility': 'PUBLIC'}},
       'email': []},
      'delegation': None,
      'external-identifiers': None,
      'keywords': None,
      'personal-details': {'credit-name': None,
       'family-name': {'value': 'Cancellieri', 'visibility': 'PUBLIC'},
       'given-names': {'value': 'Matteo', 'visibility': 'PUBLIC'},
       'other-names': None},
      'researcher-urls': {'researcher-url': [{'url': {'value': 'http://core.ac.uk'},
         'url-name': {'value': 'CORE'}},
        {'url': {'value': 'http://fosteropenscience.eu'},
         'url-name': {'value': 'FOSTER'}},
        {'url': {'value': 'http://www.matteocancellieri.

In [4]:
for profile in data['orcid-search-results']['orcid-search-result']:
    orcid = profile['orcid-profile']['orcid-identifier']['uri']
    print (orcid)

http://orcid.org/0000-0002-9558-9772


In [ ]:
class OrcidSearcher:
    def __init__(self, accessToken, writer):
        self.queue = []
        self.accessToken=accessToken
        self.writer=writer
        self.headers =  headers = {"Content-Type":"application/json",
         "Authorization": "Bearer "+self.accessToken}
    def search(self, coreId, doi):

        if (len(self.queue)<12):
            self.queue.append([self.getSearchUrl(doi),doi, coreId])
        else:
            self.queue.append(self.getSearchUrl(doi))
            requestset = (grequests.get(item[0], headers=self.headers) for item in self.queue)
            startTime = time.time()
            responses = grequests.map(requestset)
            i = 0
            for response in responses:
                if not response:
                    print(response)
                    i+=1
                    continue
                data = response.json()
                orcids=[]

                for profile in data['orcid-search-results']['orcid-search-result']:
                    orcids.append(profile['orcid-profile']['orcid-identifier']['uri'])
                if orcids:
                    print ("%d orcids found for %s doi %s"%(len(orcids), self.queue[i][2],self.queue[i][1]))
                    self.saveResults(self.queue[i][2], self.queue[i][1], orcids)
                i+=1
            endTime = time.time()
            if (endTime-startTime <1):
                    print("Ops, Hit the rate limit, sleep")
                    time.sleep(1)
            self.queue=[]

    def getSearchUrl(self, doi):
        return "https://pub.orcid.org/v1.2/search/orcid-bio/?defType=edismax&q=digital-object-ids:%22"+doi+"%22"
    def saveResults(self, coreId, doi, orcids):
        for orcid in orcids:
            row = (coreId, doi, orcid)
            self.writer.writerow(row)

    

In [9]:
import pandas
orcid_results = pandas.io.parsers.read_csv("/data/core/experiments/orcid/core_orcid_1310.csv", sep=',', names=('coreId', 'doi', 'orcid'))
orcid_results

,coreId,doi,orcid
0,18333986,'10.1080/09540962.2011.586240',http://orcid.org/0000-0003-0107-9809
1,29150961,'10.1016/j.neubiorev.2014.08.016',http://orcid.org/0000-0002-7925-4921
2,29150961,'10.1016/j.neubiorev.2014.08.016',http://orcid.org/0000-0002-2388-8466
3,18334030,'10.1093/hmg',http://orcid.org/0000-0002-1523-1106
4,18334030,'10.1093/hmg',http://orcid.org/0000-0001-8450-6196
5,18334030,'10.1093/hmg',http://orcid.org/0000-0001-8796-837X
6,18334030,'10.1093/hmg',http://orcid.org/0000-0001-5216-817X
7,18334030,'10.1093/hmg',http://orcid.org/0000-0002-3626-4468
8,18334030,'10.1093/hmg',http://orcid.org/0000-0002-2154-2242
9,18334030,'10.1093/hmg',http://orcid.org/0000-0002-6313-5119


In [11]:
len(orcid_results.doi.unique())

939461

In [12]:
len(orcid_results.orcid.unique())

214278

In [38]:
orcid_results['orcidcount'] = orcid_results.groupby("coreId")['orcid'].transform('count')
orcid_nojournals = orcid_results[orcid_results['orcidcount']<10]
orcid_nojournals = orcid_nojournals[~ orcid_nojournals.doi.str.contains("'10.(\d+)/([a-z|A-Z]+')")]
orcid_nojournals

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


,coreId,doi,orcid,orcidcount
0,18333986,'10.1080/09540962.2011.586240',http://orcid.org/0000-0003-0107-9809,1
1,29150961,'10.1016/j.neubiorev.2014.08.016',http://orcid.org/0000-0002-7925-4921,2
2,29150961,'10.1016/j.neubiorev.2014.08.016',http://orcid.org/0000-0002-2388-8466,2
13,18334016,'10.1107/S205225251302695X',http://orcid.org/0000-0002-4073-0505,2
14,18334016,'10.1107/S205225251302695X',http://orcid.org/0000-0001-6842-4289,2
15,1443577,'10.1037/0021-9010.91.1.139',http://orcid.org/0000-0001-5148-8358,1
16,6105136,'10.1162/105474601753132687',http://orcid.org/0000-0001-8783-5542,1
17,19787406,'10.1371/journal.pone.0088854',http://orcid.org/0000-0003-2985-6913,1
18,6105143,'10.2134/jeq2005.0477',http://orcid.org/0000-0002-1108-4831,1
19,20664048,'10.1136/bmjopen-2014-005524',http://orcid.org/0000-0003-4894-1689,1


In [34]:
orcid_and_repos = pandas.io.parsers.read_csv("/data/core/experiments/orcid/core_orcid_nojournal_repos1310.csv", sep=',', names=('coreId','doi', 'orcid','count', 'repo'))
orcid_and_repos

,coreId,doi,orcid,count,repo
0,18333986,'10.1080/09540962.2011.586240',http://orcid.org/0000-0003-0107-9809,1,45
1,29150961,'10.1016/j.neubiorev.2014.08.016',http://orcid.org/0000-0002-7925-4921,2,1
2,29150961,'10.1016/j.neubiorev.2014.08.016',http://orcid.org/0000-0002-2388-8466,2,1
13,18334016,'10.1107/S205225251302695X',http://orcid.org/0000-0002-4073-0505,2,45
14,18334016,'10.1107/S205225251302695X',http://orcid.org/0000-0001-6842-4289,2,45
15,1443577,'10.1037/0021-9010.91.1.139',http://orcid.org/0000-0001-5148-8358,1,50
16,6105136,'10.1162/105474601753132687',http://orcid.org/0000-0001-8783-5542,1,20
17,19787406,'10.1371/journal.pone.0088854',http://orcid.org/0000-0003-2985-6913,1,1
18,6105143,'10.2134/jeq2005.0477',http://orcid.org/0000-0002-1108-4831,1,20
19,20664048,'10.1136/bmjopen-2014-005524',http://orcid.org/0000-0003-4894-1689,1,1


In [40]:
repo_loc = pandas.io.parsers.read_csv("/data/core/experiments/orcid/repoloc.csv", sep=',')
repo_loc

,id_repository,country_code,longitude,latitude
0,0,NaN,NaN,NaN
1,1,GB,-2.099100,57.165000
2,2,GB,-2.988800,56.459900
3,3,GB,-5.935900,54.592850
4,4,GB,-1.395590,50.899680
5,5,GB,-1.211500,51.793200
6,6,GB,0.474300,51.741300
7,7,GB,-1.889600,52.486200
8,8,GB,-0.129308,51.521515
9,9,GB,-1.897168,50.742979


In [41]:
orcid_and_repoloc = pandas.merge(orcid_and_repos, repo_loc, left_on='repo',right_on="id_repository")
orcid_by_country = orcid_and_repoloc.groupby(orcid_and_repoloc.country_code).size().sort_values(ascending=False)
orcid_by_country

country_code
GB    636947
US    293433
FR    142088
AU     89070
DK     49289
DE     42426
CH     35498
ES     28388
NL     23615
BE     20121
IT     12978
PT     12244
IE      9211
IN      5290
MY      4217
FI      4172
CL      3199
JP      2636
SE      2132
NZ      2105
HU      1715
ZA      1391
NO      1216
CZ       885
TR       858
BR       572
CA       480
TZ       443
ID       350
SG       342
HR       336
GR       277
IR       242
LU       186
DZ       176
PL       136
AT       100
TW        81
QA        76
AR        70
CY        58
MX        49
PH        42
NG        27
PE        26
SI        16
UA        16
BW        15
FJ        13
CO        11
BG         5
VE         2
JM         2
SA         1
EE         1
dtype: int64

In [42]:
orcid_onlygb = orcid_and_repoloc[orcid_and_repoloc.country_code=="GB"]
orcid_onlygb

,coreId,doi,orcid,count,repo,id_repository,country_code,longitude,latitude
0,18333986,'10.1080/09540962.2011.586240',http://orcid.org/0000-0003-0107-9809,1,45,45,GB,50.737137,-3.535148
1,18334016,'10.1107/S205225251302695X',http://orcid.org/0000-0002-4073-0505,2,45,45,GB,50.737137,-3.535148
2,18334016,'10.1107/S205225251302695X',http://orcid.org/0000-0001-6842-4289,2,45,45,GB,50.737137,-3.535148
3,18333993,'10.1108/09513571311311874',http://orcid.org/0000-0003-0107-9809,1,45,45,GB,50.737137,-3.535148
4,2735318,'10.1086/490501',http://orcid.org/0000-0002-1272-3164,1,45,45,GB,50.737137,-3.535148
5,2735402,'10.1080/00467600701429030',http://orcid.org/0000-0003-3196-303X,1,45,45,GB,50.737137,-3.535148
6,2735515,'10.1063/1.2809448',http://orcid.org/0000-0003-1844-274X,1,45,45,GB,50.737137,-3.535148
7,2734625,'10.1007/s11238-007-9053-3',http://orcid.org/0000-0002-8368-7818,1,45,45,GB,50.737137,-3.535148
8,9648147,'10.1117/1.3436585',http://orcid.org/0000-0003-3367-9980,3,45,45,GB,50.737137,-3.535148
9,9648147,'10.1117/1.3436585',http://orcid.org/0000-0001-5618-0993,3,45,45,GB,50.737137,-3.535148


In [46]:
dois = len(orcid_onlygb.doi.unique())
orcids = len(orcid_onlygb.orcid.unique())
print ("%d DOIs and %d ORCIDS"%(dois, orcids))

252547 DOIs and 68441 ORCIDS


In [47]:
orcid_onlygb.groupby(orcid_onlygb.repo).size().sort_values(ascending=False)

repo
88     124256
234     77920
635     54894
289     33732
34      30935
83      30160
63      26272
647     22686
658     17586
131     14813
79      13455
59      13454
124     13255
123     11681
82      11023
132      6986
661      6876
33       6535
27       5394
695      5133
126      4754
162      4216
165      4176
128      4148
108      3996
107      3757
81       3511
80       3329
8        3198
125      3086
        ...  
55         20
110        19
253        18
15         17
166        15
26         12
638        11
686         8
78          8
134         8
104         7
117         7
620         7
169         6
119         5
96          5
99          4
149         4
505         3
38          3
92          2
167         2
170         2
683         2
138         2
40          1
103         1
21          1
24          1
3           1
dtype: int64